In [1]:
import os, shutil
import tensorflow as tf
gpu_options=tf.GPUOptions(per_process_gpu_memory_fraction=0.6)#分配20%GPU资源
config=tf.ConfigProto(gpu_options=gpu_options)
session = tf.Session(config=config)

In [2]:
from keras.preprocessing.image import ImageDataGenerator
from keras.preprocessing.image import img_to_array
from keras.preprocessing.image import load_img
import numpy as np
base_dir = './dogImages'
train_dir = os.path.join(base_dir, 'train')
validation_dir = os.path.join(base_dir, 'valid')
test_dir = os.path.join(base_dir, 'test')
train_plus_dir=os.path.join(base_dir, 'train_plus2')

Using TensorFlow backend.


# 增强数据集的操作

In [3]:
# if not os.path.exists(train_plus_dir):
#     shutil.copytree(train_dir,train_plus_dir)
# for kind in os.listdir(train_plus_dir):
#     kind_dir=os.path.join(train_plus_dir, kind)
#     if len(os.listdir(kind_dir))*4 < 5*len([x for x in os.listdir(kind_dir) if x.startswith('pic_plus')])+5:
#         continue
#     for pic in os.listdir(kind_dir):
#         pic_dir=os.path.join(kind_dir, pic)
#         if not pic.endswith(".jpg"):
#             continue
#         if pic.startswith('pic_plus'):
#             os.remove(pic_dir)
#             continue
#         print (pic_dir)
#         image = load_img(pic_dir)
#         image = img_to_array(image)
#         image = np.expand_dims(image, axis=0)
#         aug = ImageDataGenerator(rotation_range=30, 
#                                  width_shift_range=0.1,
#                                  height_shift_range=0.1, 
#                                  shear_range=0.1, 
#                                  zoom_range=0.2,
#                                  horizontal_flip=True, 
#                                  fill_mode="nearest")
#         total = 0
#         imageGen = aug.flow(image, batch_size=1, 
#                             save_to_dir=kind_dir,
#                             save_prefix="pic_plus", 
#                             save_format="jpg")
#         print (imageGen)
#         for image in imageGen:
#         # increment our counter
#             total += 1
#             # if we have reached 4 examples, break from the loop
#             if total == 4:
#                 break

In [4]:
train_dir=train_plus_dir

In [5]:
from keras import layers
from keras import models

model = models.Sequential()
model.add(layers.Conv2D(32, (3, 3), activation='relu',
                        input_shape=(150, 150, 3)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(128, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(256, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Flatten())
model.add(layers.Dense(512, activation='relu'))
model.add(layers.Dense(133, activation='softmax'))

In [6]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 148, 148, 32)      896       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 74, 74, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 72, 72, 64)        18496     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 36, 36, 64)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 34, 34, 128)       73856     
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 17, 17, 128)       0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 15, 15, 256)       295168    
__________

In [7]:
from keras import optimizers
# lr: float >= 0. 学习率 Learning rate

# momentum: float >= 0. 参数更新动量 parameter updates momentum

# decay: float >= 0. 学习率每次更新的下降率 Learning rate decay over each update

# nesterov: boolean. 是否应用 Nesterov 动量 whether to apply Nesterov momentum
model.compile(loss='categorical_crossentropy',
              optimizer=optimizers.RMSprop(lr=1e-4),
              metrics=['acc'])

In [8]:
from keras.preprocessing.image import ImageDataGenerator

# All images will be rescaled by 1./255
train_datagen = ImageDataGenerator(rescale=1./255)
validation_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
        # This is the target directory
        train_dir,
        # All images will be resized to 150x150
        target_size=(150, 150),
        batch_size=20,
        # Since we use binary_crossentropy loss, we need binary labels
        class_mode='categorical')

validation_generator = validation_datagen.flow_from_directory(
        validation_dir,
        target_size=(150, 150),
        batch_size=100,
        class_mode='categorical')

Found 33174 images belonging to 133 classes.
Found 835 images belonging to 133 classes.


In [9]:
for data_batch, labels_batch in train_generator:
    print('data batch shape:', data_batch.shape)
    print('labels batch shape:', labels_batch.shape)
    break

data batch shape: (20, 150, 150, 3)
labels batch shape: (20, 133)


In [10]:
from keras.callbacks import ModelCheckpoint   
from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True
checkpointer = ModelCheckpoint(filepath='dogskind.model.weights.best.hdf5', verbose=1, 
                               save_best_only=True)
history = model.fit_generator(train_generator,
      steps_per_epoch=100,
      epochs=50,
      validation_data=validation_generator,
      validation_steps=50, 
      callbacks=[checkpointer], 
      verbose=2)

Epoch 1/50


UnknownError: Failed to get convolution algorithm. This is probably because cuDNN failed to initialize, so try looking to see if a warning log message was printed above.
	 [[{{node conv2d_1/convolution}} = Conv2D[T=DT_FLOAT, _class=["loc:@training/RMSprop/gradients/conv2d_1/convolution_grad/Conv2DBackpropFilter"], data_format="NCHW", dilations=[1, 1, 1, 1], padding="VALID", strides=[1, 1, 1, 1], use_cudnn_on_gpu=true, _device="/job:localhost/replica:0/task:0/device:GPU:0"](training/RMSprop/gradients/conv2d_1/convolution_grad/Conv2DBackpropFilter-0-TransposeNHWCToNCHW-LayoutOptimizer, conv2d_1/kernel/read)]]
	 [[{{node metrics/acc/Mean/_93}} = _Recv[client_terminated=false, recv_device="/job:localhost/replica:0/task:0/device:CPU:0", send_device="/job:localhost/replica:0/task:0/device:GPU:0", send_device_incarnation=1, tensor_name="edge_706_metrics/acc/Mean", tensor_type=DT_FLOAT, _device="/job:localhost/replica:0/task:0/device:CPU:0"]()]]

In [ ]:
model.load_weights('dogskind.model.weights.best.hdf5')

In [ ]:
test_datagen = ImageDataGenerator(rescale=1./255)
test_generator = test_datagen.flow_from_directory(
        test_dir,
        target_size=(150, 150),
        batch_size=20,
        class_mode='categorical')

model.evaluate_generator(test_generator,steps=17, max_queue_size=50, workers=1, use_multiprocessing=True, verbose=0)

In [ ]:
import matplotlib.pyplot as plt
acc = history.history['acc']
val_acc = history.history['val_acc']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(len(acc))

plt.plot(epochs, acc, 'bo', label='Training acc')
plt.plot(epochs, val_acc, 'b', label='Validation acc')
plt.title('Training and validation accuracy')
plt.legend()

plt.figure()

plt.plot(epochs, loss, 'bo', label='Training loss')
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.legend()

plt.show()